In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm, trange
from screaming_rocks import RATE, SENSOR_IDS
from screaming_rocks import dask_file_reading as dfr

/Users/tlingard/anaconda3/lib/python3.6/site-packages/dask/dataframe/utils.py:15: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
# Update this with the location to your own data
data_folder = '/Volumes/data8pmb/EB2/SRM/'

We can eaily read small amounts of data as numpy arrays - here we read one second of data from the Sensor with ID 90414, starting at minute 25

In [4]:
print(dfr.read_batch(90414, 25 * 60, 1, data_folder=data_folder))

[[33531 32771 32599 32756]
 [33529 32757 32602 32789]
 [33530 32710 32705 32794]
 ...
 [33515 32853 32857 32756]
 [33534 32851 32842 32694]
 [33494 32898 32754 32694]]


We can also read this in as a Pandas DataFrame, with correct time series indexing and columns

In [5]:
dfr.read_batch_as_pandas(90414, 25 * 60, 1, data_folder=data_folder)

90414                     
                     0      1      2      3
00:25:00         33531  32771  32599  32756
00:25:00.000000  33529  32757  32602  32789
00:25:00.000000  33530  32710  32705  32794
00:25:00.000000  33552  32675  32796  32820
00:25:00.000000  33491  32746  32823  32817
...                ...    ...    ...    ...
00:25:00.999999  33482  32834  32897  32737
00:25:00.999999  33513  32869  32829  32731
00:25:00.999999  33515  32853  32857  32756
00:25:00.999999  33534  32851  32842  32694
00:25:00.999999  33494  32898  32754  32694

[10000000 rows x 4 columns]

If we want to read more data, we need to use Dask to handle out-of-memory computation, which is made easy using `dfr.read_as_dask`.

I.e. if we want to read two minutes of data, starting at minute 20 (reading 8GB worth of files, batching into pandas DataFrames of one second):

In [6]:
dfr.read_as_dask(90414, 20 * 60, 2 * 60, batch_size=2, data_folder=data_folder)

Dask DataFrame Structure:
                 90414                        
                     0       1       2       3
npartitions=60                                
                uint16  uint16  uint16  uint16
                   ...     ...     ...     ...
...                ...     ...     ...     ...
                   ...     ...     ...     ...
                   ...     ...     ...     ...
Dask Name: from-delayed, 120 tasks

We then can execute lazy computations on this data (warning, this takes a LONG time):

In [7]:
# dfr.read_as_dask(90414, 20 * 60, 2 * 60, batch_size=2, data_folder=data_folder).mean(axis=1).compute()